In [ ]:
pip install emnist

ERROR: Operation cancelled by user


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from emnist import list_datasets, extract_training_samples, extract_test_samples

ModuleNotFoundError: ignored

In [ ]:
# load emnist dataset 'by class'

images, labels = extract_training_samples('byclass')
images_test, labels_test = extract_test_samples('byclass')

In [ ]:
# find index of uppercase letters

tr_index_UPP = [i for i in range(len(labels)) if labels[i]>=10 and labels[i] <= 35] 
ts_index_UPP = [i for i in range(len(labels_test)) if labels_test[i]>=10 and labels_test[i] <= 35]

In [ ]:
# save images and labels in uppercase

im_train_UPP = images[tr_index_UPP]
im_test_UPP = images_test[ts_index_UPP]
lab_train_UPP = labels[tr_index_UPP]
lab_test_UPP = labels_test[ts_index_UPP]

In [ ]:
# reformulate labels for application of built-in function 'to_categorical'

alf_UPP = {
       10: 0,
       11: 1,
       12: 2,
       13: 3,
       14: 4,
       15: 5,
       16: 6,
       17: 7,
       18: 8,
       19: 9,
       20: 10,
       21: 11,
       22: 12,
       23: 13,
       24: 14,
       25: 15,
       26: 16,
       27: 17,
       28: 18,
       29: 19,
       30: 20,
       31: 21,
       32: 22,
       33: 23,
       34: 24,
       35: 25,
       }

lab_train_UPP = [alf_UPP[label] for label in lab_train_UPP]
lab_test_UPP = [alf_UPP[label] for label in lab_test_UPP]

In [ ]:
# check labels

lab_train_UPP[0]

In [ ]:
# define image width and height and batch size

# Batch size for training and validation
batch_size = 32

# Desired image dimensions
img_width = 28
img_height = 28

In [ ]:
# split training set in traininig and validation set

def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid

In [ ]:
# Splitting data into training and validation sets
x_train_UPP, x_valid_UPP, y_train_UPP, y_valid_UPP = split_data(np.array(im_train_UPP), np.array(lab_train_UPP))

In [ ]:
# to categorical 

y_train_UPP = tf.keras.utils.to_categorical(y_train_UPP, 26)
y_valid_UPP = tf.keras.utils.to_categorical(y_valid_UPP, 26)
lab_test_UPP = tf.keras.utils.to_categorical(lab_test_UPP, 26)

In [ ]:
# normalize training, validation and test data

x_train_UPP = tf.keras.utils.normalize(x_train_UPP, axis=1)
x_valid_UPP = tf.keras.utils.normalize(x_valid_UPP, axis=1)
im_test_UPP = tf.keras.utils.normalize(im_test_UPP, axis=1)

In [ ]:
# build model

def build_model():
    # Inputs to the model
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="image", dtype="float32"
    )

    # Output layer
    x = layers.Dense(units=256, activation='sigmoid', name='dense_1')(input_img)
    x = layers.Dense(units=128, activation="sigmoid", name="dense2")(x)
    x = layers.Dense(units=64, activation="sigmoid", name="dense3")(x)
    x = layers.Flatten()(x)
    output = layers.Dense(units=26, activation="softmax", name="dense4")(x)

    # Define the model
    model = keras.models.Model(
        inputs=input_img, outputs=output, name="ocr_model_v1"
    )
    # Optimizer
    opt = keras.optimizers.Adam()
    # Compile the model and return
    model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy'])

    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='/content/drive/My Drive/TESI/Models/Immagini/model5_byclass_upper_noDA.png')

In [ ]:
epochs = 100
early_stopping_patience = 10
# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=early_stopping_patience, restore_best_weights=True, verbose=1
)

# Model CheckPoint
cp = keras.callbacks.ModelCheckpoint('/content/drive/My Drive/TESI/Models/model5_UPP_byclass_noDA.h5',
                                     verbose=1,
                                     monitor='val_accuracy',
                                     save_best_only=True,
                                     mode='max')

In [ ]:
# fit model
history1_UPP = model.fit(x_train_UPP,y_train_UPP,
                    validation_data=(x_valid_UPP,y_valid_UPP),
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=[cp, early_stopping])

In [ ]:
plt.figure(figsize=(16,9))
# summarize history for accuracy
plt.plot(history1_UPP.history['accuracy'])
plt.plot(history1_UPP.history['val_accuracy'])
plt.title('Model 5 uppercase letters accuracy (without Data Augmentation)')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
plt.figure(figsize=(16,9))

# summarize history for loss
plt.plot(history1_UPP.history['loss'])
plt.plot(history1_UPP.history['val_loss'])
plt.title('Model 5 uppercase letters loss (without Data Augmentation)')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
evaluation = model.evaluate(im_test_UPP, lab_test_UPP)

In [ ]:
evaluation